In [2]:
import numpy as np
# import cPickle as pickle
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import math

# from modelAny import *

from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import embedding_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import rnn
from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import variable_scope

In [3]:
import gym
env = gym.make('CartPole-v0')

In [4]:
# hyperparameters
H = 8 # number of hidden layer neurons
learning_rate = 1e-2
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?

model_bs = 3 # Batch size when learning from model
real_bs = 3 # Batch size when learning from real environment

# model initialization
D = 4 # input dimensionality

In [5]:
tf.reset_default_graph()
observations = tf.placeholder(tf.float32, [None,4] , name="input_x")
W1 = tf.get_variable("W1", shape=[4, H],
           initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W2 = tf.get_variable("W2", shape=[H, 1],
           initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(layer1,W2)
probability = tf.nn.sigmoid(score)

tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal")
adam = tf.train.AdamOptimizer(learning_rate=learning_rate)
W1Grad = tf.placeholder(tf.float32,name="batch_grad1")
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages) 
newGrads = tf.gradients(loss,tvars)
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [6]:
mH = 256 # model layer size

input_data = tf.placeholder(tf.float32, [None, 5])
with tf.variable_scope('rnnlm'):
    softmax_w = tf.get_variable("softmax_w", [mH, 50])
    softmax_b = tf.get_variable("softmax_b", [50])

previous_state = tf.placeholder(tf.float32, [None,5] , name="previous_state")
W1M = tf.get_variable("W1M", shape=[5, mH],
           initializer=tf.contrib.layers.xavier_initializer())
B1M = tf.Variable(tf.zeros([mH]),name="B1M")
layer1M = tf.nn.relu(tf.matmul(previous_state,W1M) + B1M)
W2M = tf.get_variable("W2M", shape=[mH, mH],
           initializer=tf.contrib.layers.xavier_initializer())
B2M = tf.Variable(tf.zeros([mH]),name="B2M")
layer2M = tf.nn.relu(tf.matmul(layer1M,W2M) + B2M)
wO = tf.get_variable("wO", shape=[mH, 4],
           initializer=tf.contrib.layers.xavier_initializer())
wR = tf.get_variable("wR", shape=[mH, 1],
           initializer=tf.contrib.layers.xavier_initializer())
wD = tf.get_variable("wD", shape=[mH, 1],
           initializer=tf.contrib.layers.xavier_initializer())

bO = tf.Variable(tf.zeros([4]),name="bO")
bR = tf.Variable(tf.zeros([1]),name="bR")
bD = tf.Variable(tf.ones([1]),name="bD")


predicted_observation = tf.matmul(layer2M,wO,name="predicted_observation") + bO
predicted_reward = tf.matmul(layer2M,wR,name="predicted_reward") + bR
predicted_done = tf.sigmoid(tf.matmul(layer2M,wD,name="predicted_done") + bD)

true_observation = tf.placeholder(tf.float32,[None,4],name="true_observation")
true_reward = tf.placeholder(tf.float32,[None,1],name="true_reward")
true_done = tf.placeholder(tf.float32,[None,1],name="true_done")


predicted_state = tf.concat([predicted_observation,predicted_reward,predicted_done], 1)

observation_loss = tf.square(true_observation - predicted_observation)

reward_loss = tf.square(true_reward - predicted_reward)

done_loss = tf.multiply(predicted_done, true_done) + tf.multiply(1-predicted_done, 1-true_done)
done_loss = -tf.log(done_loss)

model_loss = tf.reduce_mean(observation_loss + done_loss + reward_loss)

modelAdam = tf.train.AdamOptimizer(learning_rate=learning_rate)
updateModel = modelAdam.minimize(model_loss)

In [11]:
def resetGradBuffer(gradBuffer):
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    return gradBuffer
        
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r


# This function uses our model to produce a new state when given a previous state and action
def stepModel(sess, xs, action):
    toFeed = np.reshape(np.hstack([xs[-1][0],np.array(action)]),[1,5])
    myPredict = sess.run([predicted_state],feed_dict={previous_state: toFeed})
    reward = myPredict[0][:,4]
    observation = myPredict[0][:,0:4]
    observation[:,0] = np.clip(observation[:,0],-2.4,2.4)
    observation[:,2] = np.clip(observation[:,2],-0.4,0.4)
    doneP = np.clip(myPredict[0][:,5],0,1)
    if doneP > 0.1 or len(xs)>= 300:
        done = True
    else:
        done = False
    return observation, reward, done

In [12]:
xs,drs,ys,ds = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
real_episodes = 1
init = tf.initialize_all_variables()
batch_size = real_bs

drawFromModel = False # When set to True, will use model for observations
trainTheModel = True # Whether to train the model
trainThePolicy = False # Whether to train the policy
switch_point = 1

# Launch the graph
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset()
    x = observation
    gradBuffer = sess.run(tvars)
    gradBuffer = resetGradBuffer(gradBuffer)
    
    while episode_number <= 5000:
        # Start displaying environment once performance is acceptably high.
        if (reward_sum/batch_size > 150 and drawFromModel == False) or rendering == True : 
            env.render()
            rendering = True
            
        x = np.reshape(observation,[1,4])

        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0

        # record various intermediates (needed later for backprop)
        xs.append(x) 
        y = 1 if action == 0 else 0 
        ys.append(y)
        
        # step the  model or real environment and get new measurements
        if drawFromModel == False:
            observation, reward, done, info = env.step(action)
        else:
            observation, reward, done = stepModel(sess,xs,action)
                
        reward_sum += reward
        
        ds.append(done*1)
        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        if done: 
            
            if drawFromModel == False: 
                real_episodes += 1
            episode_number += 1

            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            epd = np.vstack(ds)
            xs,drs,ys,ds = [],[],[],[] # reset array memory
            
            if trainTheModel == True:
                actions = np.array([np.abs(y-1) for y in epy][:-1])
                state_prevs = epx[:-1,:]
                state_prevs = np.hstack([state_prevs,actions])
                state_nexts = epx[1:,:]
                rewards = np.array(epr[1:,:])
                dones = np.array(epd[1:,:])
                state_nextsAll = np.hstack([state_nexts,rewards,dones])

                feed_dict={previous_state: state_prevs, true_observation: state_nexts,true_done:dones,true_reward:rewards}
                loss,pState,_ = sess.run([model_loss,predicted_state,updateModel],feed_dict)
            if trainThePolicy == True:
                discounted_epr = discount_rewards(epr).astype('float32')
                discounted_epr -= np.mean(discounted_epr)
                discounted_epr /= np.std(discounted_epr)
                tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
                
                # If gradients becom too large, end training process
                if np.sum(tGrad[0] == tGrad[0]) == 0:
                    break
                for ix,grad in enumerate(tGrad):
                    gradBuffer[ix] += grad
                
            if switch_point + batch_size == episode_number: 
                switch_point = episode_number
                if trainThePolicy == True:
                    sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                    gradBuffer = resetGradBuffer(gradBuffer)

                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                if drawFromModel == False:
                    print('World Perf: Episode %f. Reward %f. action: %f. mean reward %f.' % (real_episodes,reward_sum/real_bs,action, running_reward/real_bs))
                    if reward_sum/batch_size > 200:
                        break
                reward_sum = 0

                # Once the model has been trained on 100 episodes, we start alternating between training the policy
                # from the model and training the model from the real environment.
#                 if episode_number > 100:
#                     drawFromModel = not drawFromModel
#                     trainTheModel = not trainTheModel
#                     trainThePolicy = not trainThePolicy
            
            if drawFromModel == True:
                observation = np.random.uniform(-0.1,0.1,[4]) # Generate reasonable starting point
                batch_size = model_bs
            else:
                observation = env.reset()
                batch_size = real_bs
                
print(real_episodes)

World Perf: Episode 4.000000. Reward 26.666667. action: 1.000000. mean reward 26.666667.
World Perf: Episode 7.000000. Reward 30.666667. action: 1.000000. mean reward 26.706667.
World Perf: Episode 10.000000. Reward 27.000000. action: 1.000000. mean reward 26.709600.
World Perf: Episode 13.000000. Reward 26.000000. action: 0.000000. mean reward 26.702504.
World Perf: Episode 16.000000. Reward 35.333333. action: 1.000000. mean reward 26.788812.
World Perf: Episode 19.000000. Reward 30.000000. action: 0.000000. mean reward 26.820924.
World Perf: Episode 22.000000. Reward 28.666667. action: 1.000000. mean reward 26.839382.
World Perf: Episode 25.000000. Reward 21.333333. action: 1.000000. mean reward 26.784321.
World Perf: Episode 28.000000. Reward 20.666667. action: 0.000000. mean reward 26.723145.
World Perf: Episode 31.000000. Reward 13.000000. action: 0.000000. mean reward 26.585913.
World Perf: Episode 34.000000. Reward 15.666667. action: 0.000000. mean reward 26.476721.
World Perf: 

World Perf: Episode 280.000000. Reward 29.333333. action: 0.000000. mean reward 53.057018.
World Perf: Episode 283.000000. Reward 34.000000. action: 0.000000. mean reward 54.176697.
World Perf: Episode 286.000000. Reward 27.333333. action: 0.000000. mean reward 56.108547.
World Perf: Episode 289.000000. Reward 54.333333. action: 0.000000. mean reward 55.888367.
World Perf: Episode 292.000000. Reward 12.000000. action: 1.000000. mean reward 54.978394.
World Perf: Episode 295.000000. Reward 50.333333. action: 1.000000. mean reward 54.641659.
World Perf: Episode 298.000000. Reward 13.333333. action: 0.000000. mean reward 54.207981.
World Perf: Episode 301.000000. Reward 23.000000. action: 0.000000. mean reward 55.727848.
World Perf: Episode 304.000000. Reward 29.000000. action: 0.000000. mean reward 55.101505.
World Perf: Episode 307.000000. Reward 66.000000. action: 1.000000. mean reward 54.919590.
World Perf: Episode 310.000000. Reward 36.666667. action: 0.000000. mean reward 62.533070.

World Perf: Episode 556.000000. Reward 86.333333. action: 1.000000. mean reward 80.348656.
World Perf: Episode 559.000000. Reward 62.333333. action: 1.000000. mean reward 81.857132.
World Perf: Episode 562.000000. Reward 89.666667. action: 1.000000. mean reward 81.504982.
World Perf: Episode 565.000000. Reward 143.666667. action: 1.000000. mean reward 83.817589.
World Perf: Episode 568.000000. Reward 54.333333. action: 0.000000. mean reward 82.833023.
World Perf: Episode 571.000000. Reward 120.000000. action: 1.000000. mean reward 82.778038.
World Perf: Episode 574.000000. Reward 59.333333. action: 0.000000. mean reward 81.974899.
World Perf: Episode 577.000000. Reward 52.333333. action: 1.000000. mean reward 83.904434.
World Perf: Episode 580.000000. Reward 73.333333. action: 1.000000. mean reward 83.133347.
World Perf: Episode 583.000000. Reward 125.000000. action: 0.000000. mean reward 82.839943.
World Perf: Episode 586.000000. Reward 87.000000. action: 1.000000. mean reward 85.5259

World Perf: Episode 823.000000. Reward 154.000000. action: 1.000000. mean reward 137.041428.
World Perf: Episode 826.000000. Reward 178.666667. action: 1.000000. mean reward 136.649124.
World Perf: Episode 829.000000. Reward 188.333333. action: 0.000000. mean reward 136.369354.
World Perf: Episode 832.000000. Reward 167.333333. action: 0.000000. mean reward 136.077423.
World Perf: Episode 835.000000. Reward 177.666667. action: 0.000000. mean reward 136.086380.
World Perf: Episode 838.000000. Reward 181.000000. action: 0.000000. mean reward 135.544907.
World Perf: Episode 841.000000. Reward 140.000000. action: 1.000000. mean reward 134.994034.
World Perf: Episode 844.000000. Reward 200.000000. action: 1.000000. mean reward 135.044754.
World Perf: Episode 847.000000. Reward 200.000000. action: 1.000000. mean reward 134.976181.
World Perf: Episode 850.000000. Reward 177.333333. action: 1.000000. mean reward 136.170700.
World Perf: Episode 853.000000. Reward 165.333333. action: 0.000000. m

World Perf: Episode 1087.000000. Reward 200.000000. action: 1.000000. mean reward 161.652756.
World Perf: Episode 1090.000000. Reward 200.000000. action: 1.000000. mean reward 161.443008.
World Perf: Episode 1093.000000. Reward 200.000000. action: 1.000000. mean reward 163.164688.
World Perf: Episode 1096.000000. Reward 200.000000. action: 0.000000. mean reward 164.540756.
World Perf: Episode 1099.000000. Reward 200.000000. action: 1.000000. mean reward 166.056992.
World Perf: Episode 1102.000000. Reward 200.000000. action: 1.000000. mean reward 167.523438.
World Perf: Episode 1105.000000. Reward 200.000000. action: 1.000000. mean reward 169.166565.
World Perf: Episode 1108.000000. Reward 200.000000. action: 1.000000. mean reward 170.043915.
World Perf: Episode 1111.000000. Reward 189.000000. action: 0.000000. mean reward 171.524368.
World Perf: Episode 1114.000000. Reward 200.000000. action: 0.000000. mean reward 172.356491.
World Perf: Episode 1117.000000. Reward 200.000000. action: 

World Perf: Episode 1351.000000. Reward 200.000000. action: 0.000000. mean reward 179.964340.
World Perf: Episode 1354.000000. Reward 176.000000. action: 0.000000. mean reward 178.336075.
World Perf: Episode 1357.000000. Reward 200.000000. action: 1.000000. mean reward 178.248779.
World Perf: Episode 1360.000000. Reward 200.000000. action: 1.000000. mean reward 178.696289.
World Perf: Episode 1363.000000. Reward 200.000000. action: 0.000000. mean reward 177.284592.
World Perf: Episode 1366.000000. Reward 200.000000. action: 0.000000. mean reward 176.093872.
World Perf: Episode 1369.000000. Reward 200.000000. action: 0.000000. mean reward 178.014511.
World Perf: Episode 1372.000000. Reward 200.000000. action: 0.000000. mean reward 176.608261.
World Perf: Episode 1375.000000. Reward 200.000000. action: 1.000000. mean reward 177.300964.
World Perf: Episode 1378.000000. Reward 200.000000. action: 0.000000. mean reward 178.744812.
World Perf: Episode 1381.000000. Reward 200.000000. action: 

KeyboardInterrupt: 